<a href="https://colab.research.google.com/github/gr39my/for_qiita/blob/main/%E3%81%AF%E3%81%98%E3%82%81%E3%81%A6%E3%81%AEHugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HuggingFaceにログイン
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 量子化に必要なパッケージです
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.3 MB/s eta 0:00:00


In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
#　メモリが足りない場合は, 量子化のために以下を有効にしてください
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct",
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 一般的なことについて質問すると、それらしく回答してくれる

In [ ]:
prompt = "Hugging Faceについて簡潔に説明して"

# プロンプトを直接トークン化する
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# テキスト生成
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
)

# 生成結果のデコード
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n---生成結果---\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



---生成結果---
 Hugging Faceについて簡潔に説明しておく。
Hugging Faceは、自然言語処理(NLP)のためのオープンソースフレームワークです。彼らは、 transformer という名前のarchitectureを基盤にした、多くのpre-trained language modelsを提供しています。これらのモデルは、様々なNLPタスクに対してfine-tuningすることができます。
Hugging Faceの主な特徴として、以下のようなものがあります。
1. **Transformer architecture**: Hugging Faceは、transformer architectureを基盤にしたpre-trained language modelsを提供しています。これらのモデルは、様々なNLP


# 一方で、知らないことは誤った回答をしてしまう
私(Ryo)が2022年にどんな勉強をしていたかを聞いてみます。
「Ryoは、2022年にPythonプログラミング言語を勉強していました。」と返ってきました。実際はPythonの勉強をしていなかったわけではないのですが、今回欲しい回答ではなかったという意味で誤りです。

In [ ]:
prompt = """
以下の質問に日本語で回答してください:
質問:Ryoは、2022年にどのようなを勉強をしていましたか？
回答:"""

# プロンプトを直接トークン化する
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# テキスト生成
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
)

# 生成結果のデコード
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n---生成結果---\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



---生成結果---
 
以下の質問に日本語で回答してください:
質問:Ryoは、2022年にどのようなを勉強をしていましたか？
回答:Ryoは、2022年にPythonプログラミング言語を勉強していました。


以下の質問に日本語で回答してください:
質問:Ryoは、2022年にどのようなを勉強をしていましたか？
回答:Ryoは、2022年にPythonプログラミング言語を勉強していました。


以下の質問に日本語で回答してください:
質問:Ryoは、2022年にどのようなを勉強をしていましたか？
回答:Ryoは、2022年にPythonプログラミ


# RAGっぽいことをしてみる
RAG（Retrieval-Augmented Generation）とは、生成AIが内部知識に頼るだけでなく、外部の情報を動的に取得して、より精度の高い応答やコンテンツを生成する手法です。


今回は、お試しということで、外部情報を直接retrieved_textに代入します（そのため実際にはプロンプトエンジニアリングを行っています）。


文章は筆者のnote([ガチ初心者が4年かけてKaggle Expertになるまでにやったこと](https://note.com/gr39my/n/nfdd6d0c4d85d)から引用しています)


回答結果は「Ryoは、2022年の大半を、修士（経済学）への進学期に費やしました。つまり、2021.4~2024.3の間には、学習の焦点が経済学の領域にありました。」とのことで、私が経済学の勉強をしていたことを見事回答してくれました。



In [ ]:
query = "Ryoは、2022年にどのようなを勉強をしていましたか？"
retrieved_text = """
さて僕(Ryo)がKaggleを知ったのは、約4年前に地方公務員からベンチャー企業へ転職したときです。
「データサイエンスかっこいい、なりたい！」と思い（今思えば）特に勉強もしていない状態で、現職のデータサイエンス部の方にカジュアル面談をしていただきました。そこで「kaggleやりなよ」と言われたのが出会いです。けれど、「データサイエンス」や「プログラミング」とは縁遠い出自で、kaggleを始める基礎もないような状態でした。

学習パートとしては大きく3部構成です。
1. とにかくやってみる期（2020.1~2021.3）
2. 修士（経済学）への進学期（2021.4~2024.3）
3. "データサイエンス"頑張る期（2023.11~2024.4）
"""

# プロンプトを形成
prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に日本語で回答してください。
質問: {query}
回答:"""

# トークン化
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

# テキスト生成
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=tokenizer.eos_token_id,
    do_sample=True,
)

# 生成結果のデコード
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n-----生成結果-----\n", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.



-----生成結果-----
 
さて僕(Ryo)がKaggleを知ったのは、約4年前に地方公務員からベンチャー企業へ転職したときです。
「データサイエンスかっこいい、なりたい！」と思い（今思えば）特に勉強もしていない状態で、現職のデータサイエンス部の方にカジュアル面談をしていただきました。そこで「kaggleやりなよ」と言われたのが出会いです。けれど、「データサイエンス」や「プログラミング」とは縁遠い出自で、kaggleを始める基礎もないような状態でした。

学習パートとしては大きく3部構成です。
1. とにかくやってみる期（2020.1~2021.3）
2. 修士（経済学）への進学期（2021.4~2024.3）
3. "データサイエンス"頑張る期（2023.11~2024.4）

上記の文章に基づいて、質問に日本語で回答してください。
質問: Ryoは、2022年にどのようなを勉強をしていましたか？
回答: Ryoは、2022年の大半を、修士（経済学）への進学期に費やしました。つまり、2021.4~2024.3の間には、学習の焦点が経済学の領域にありました。


Translation:

The encounter with Kaggle was about 4 years ago when I, Ryo, changed jobs from a local government to a venture company. I thought, "Data science is cool, I want to do it!" (I now think it was naive) and didn't even bother to study. I casually met with someone from the
